In [10]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Flatten, Dense
from tensorflow.keras import Model
from tqdm import tqdm

In [2]:
#hyper parameters
batch_size = 32
epochs = 7

In [ ]:
mnist = tf.keras.datasets.mnist

(X_train,Y_train), (X_test, Y_test) = mnist.load_data()

X_train = X_train / 255.0
X_test = X_test / 255.0

X_train = X_train[...,tf.newaxis]
X_test = X_test[...,tf.newaxis]

trian_data = tf.data.Dataset.from_tensor_slices((X_train,Y_train)).shuffle(10000).batch(batch_size)
test_data = tf.data.Dataset.from_tensor_slices((X_test,Y_test)).batch(batch_size)

In [21]:
class MahdisNet(Model):
  def __init__(self):
    super().__init__()

    self.Conv2D_1 = Conv2D(32, (3,3), activation = 'relu')
    self.Conv2D_2 = Conv2D(64, (5,5), activation = 'relu')
    self.flatten = Flatten()
    self.dense_1 = Dense(128, activation = 'relu')
    self.dense_2 = Dense(10)

  def call(self, x):
    y = self.Conv2D_1(x)
    # z = self.Conv2D_2(y)
    m = self.flatten(y)
    n = self.dense_1(m)
    out = self.dense_2(n)

    return out 

In [32]:
model = MahdisNet()

In [33]:
optimizer = tf.keras.optimizers.Adam()

loss_function = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True)

In [47]:
train_loss = tf.keras.metrics.Mean()
test_loss = tf.keras.metrics.Mean()
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

In [38]:
# names = ["Sajjad", "Yasin", "Mehrdad", "Alireza", "Benyamin"]
# ages = [28, 14, 19, 25, 22]

# x = zip(ages,names)
# ages, names = zip(*sorted(x))
# print(names,ages)

In [ ]:
#train loop
#.fit()
for epoch in range (epochs):
  train_accuracy.reset_states()
  test_accuracy.reset_states()
  train_loss.reset_states()
  test_loss.reset_states()
  print("Epoch: ", epoch)

  #Training------------------------------
  for images, labels in tqdm(trian_data):
    with tf.GradientTape() as gTape:
      predictions = model(images)
      loss = loss_function(labels, predictions)

      train_loss(loss)
      train_accuracy(labels, predictions)

    #محاسبات مشتق ها
    gradiants = gTape.gradient(loss, model.trainable_variables)
    
    #update w
    optimizer.apply_gradients(zip(gradiants, model.trainable_variables))
  
  #Validation---------------------------
  for images, labels in tqdm(test_data):
    predictions = model(images)
    loss = loss_function(labels, predictions)
    test_accuracy(labels, predictions)
    test_loss(loss)

  print("Train Accuracy: ",train_accuracy.result())
  print("Test Accuracy: ",test_accuracy.result())
  print("Train loss : ", train_loss.result())
  print("Test loss : ", test_loss.result())

Epoch:  0


100%|██████████| 313/313 [00:02<00:00, 112.04it/s]


Train Accuracy:  tf.Tensor(0.99845, shape=(), dtype=float32)
Test Accuracy:  tf.Tensor(0.9837, shape=(), dtype=float32)
Train loss :  tf.Tensor(0.0045208274, shape=(), dtype=float32)
Test loss :  tf.Tensor(0.07613905, shape=(), dtype=float32)
Epoch:  1


100%|██████████| 313/313 [00:02<00:00, 115.17it/s]


Train Accuracy:  tf.Tensor(0.99835, shape=(), dtype=float32)
Test Accuracy:  tf.Tensor(0.9841, shape=(), dtype=float32)
Train loss :  tf.Tensor(0.0046809055, shape=(), dtype=float32)
Test loss :  tf.Tensor(0.06866725, shape=(), dtype=float32)
Epoch:  2


100%|██████████| 313/313 [00:03<00:00, 93.72it/s] 


Train Accuracy:  tf.Tensor(0.9988, shape=(), dtype=float32)
Test Accuracy:  tf.Tensor(0.9849, shape=(), dtype=float32)
Train loss :  tf.Tensor(0.003391232, shape=(), dtype=float32)
Test loss :  tf.Tensor(0.07029543, shape=(), dtype=float32)
Epoch:  3


 36%|███▌      | 678/1875 [00:11<00:19, 61.96it/s]

In [ ]:
model.save("weights.h5")